# Final Project
## 1. Read the datasets

In [1]:
import pandas as pd
import numpy as np
np.random.seed(30224)

In [2]:
# Read train1.csv and train2.csv
train1 = pd.read_csv("./../data/train1.csv", header=0)
train2 = pd.read_csv("./../data/train2.csv", header=0)

In [3]:
train1.head(5)

,headline,clickbait
0,Should I Get Bings,1
1,Which TV Female Friend Group Do You Belong In,1
2,"The New ""Star Wars: The Force Awakens"" Trailer...",1
3,"This Vine Of New York On ""Celebrity Big Brothe...",1
4,A Couple Did A Stunning Photo Shoot With Their...,1


In [4]:
train2.head(5)

,label,title
0,news,China and Economic Reform: Xi Jinping’s Track ...
1,news,Trade to Be a Big Topic in Theresa May’s U.S. ...
2,clickbait,"The Top Beaches In The World, According To Nat..."
3,clickbait,Sheriff’s Report Provides New Details on Tamir...
4,news,Surgeon claiming he will transplant volunteer'...


In [5]:
# Check for missing data
train1[train1.isna().any(axis=1)]

,headline,clickbait


In [6]:
train2[train2.isna().any(axis=1)]

,label,title


In [7]:
# Combine datasets
train2["clickbait"] = np.where(train2["label"] == "clickbait", 1, 0)
train2["headline"] = train2["title"]

df = pd.concat([train1, train2], join="inner")

In [12]:
# Show number of clickbait vs non-clickbait titles in the dataset
df["clickbait"].value_counts()

0    32739
1    20290
Name: clickbait, dtype: int64

## 2. Exploratory data analysis